In [ ]:
LOCAL_FOLDER_BQ="nyc_taxi_data_bq"
if not os.path.exists(LOCAL_FOLDER_BQ):
    os.makedirs(LOCAL_FOLDER_BQ)

Creation de la table et de la base de données

In [ ]:
import sqlite3
conn = sqlite3.connect('raw_yellowtrips.db')
cur = conn.cursor()
cur.execute('''
CREATE TABLE IF NOT EXISTS yellow_trips (
    VendorID INTEGER,
    tpep_pickup_datetime TEXT,
    tpep_dropoff_datetime TEXT,
    passenger_count REAL,
    trip_distance REAL,
    RatecodeID REAL,
    store_and_fwd_flag TEXT,
    PULocationID INTEGER,
    DOLocationID INTEGER,
    payment_type INTEGER,
    fare_amount REAL,
    extra REAL,
    mta_tax REAL,
    tip_amount REAL,
    tolls_amount REAL,
    improvement_surcharge REAL,
    total_amount REAL,
    congestion_surcharge REAL,
    airport_fee REAL,
    source_file TEXT
)
''')

conn.commit()

Liste des fichiers à integrer dans la table bigquery

In [ ]:
fichiers = glob.glob(os.path.join(LOCAL_FOLDER, '*yellow*.parquet'))

Selection des colonnes communes 

In [ ]:
ref=pd.read_parquet("/content/nyc_taxi_data/yellow_tripdata_2024-01.parquet")
entetesRef=ref.columns

In [ ]:
conn = sqlite3.connect('raw_yellowtrips.db')
cur = conn.cursor()

for fichier in fichiers:
    nom_fichier = os.path.basename(fichier)

    # Vérifier si le fichier a déjà été inséré
    cur.execute('SELECT 1 FROM yellow_trips WHERE source_file = ?', (nom_fichier,))
    existe = cur.fetchone()

    if not existe:
        print(f"Insertion de {nom_fichier}...")
        df = pd.read_parquet(fichier)
        df=df[entetesRef].copy(deep=True)

        # Ajouter une colonne source_file
        df['source_file'] = nom_fichier

        # Insérer dans la base
        df.to_sql('yellow_trips', conn, if_exists='append', index=False)
    else:
        print(f"{nom_fichier} déjà présent. Ignoré.")

conn.commit()
conn.close()